<a href="https://colab.research.google.com/github/jiiwon129/ESAA/blob/main/OB/%EC%97%B0%EC%8A%B5%20%EB%AC%B8%EC%A0%9C/09_19_%EC%84%B8%EC%85%98_%EC%95%99%EC%83%81%EB%B8%94_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **앙상블 학습과 랜덤 포레스트 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch07 앙상블 학습과 랜덤 포레스트 연습문제 2, 7, 8, 9번
- 이론적 지식을 묻는 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

In [1]:
# import libraries
import numpy as np

### **1. 직접 투표와 간접 투표 분류기 사이의 차이점은 무엇일까요?**
___

1. 직접 투표(Hard Voting)

: 각 분류기의 예측을 모아서 가장 많이 선택된 클래스를 예측하는 것 (즉, 다수결 투표)

2. 간접 투표(Soft Voting)

: 모든 분류기가 클래스의 확률을 예측할 수 있다면, (즉, 분류기에 predict_proba() 메서드가 존재) 개별 분류기의 예측을 평균 내어 확률이 가장 높은 클래스를 예측 가능

간접 투표의 경우, 확률이 높은 투표에 비중을 더 두기 때문에 직접 투표 방식보다 성능이 높다는 장점 존재

### **2. 그레디언트 부스팅 앙상블이 훈련 데이터에 과대 적합되었다면 학습률을 어떻게 해야 할까요?**
___

그레디언트 부스팅에서 학습률(learning_rate)은 앙상블에 추가되는 각 트리의 예측 기여도를 조절하는 역할
- 학습률이 높을 수록, 훈련 데이터의 노이즈까지 학습하게 만들어 과대적합을 유발

그레디언트 부스팅 앙상블이 과대적합 되었다는 것은 높은 학습률을 가지고 예측을 진행했음을 의미
- 따라서, 학습률을 낮추는 방향으로 진행

하지만 학습률을 낮추면 학습 양이 줄어들기 때문에, 트리의 개수를 설정하는 매개변수인 n_estimators를 증가시켜야함

--> 즉, 학습률을 낮추며 트리의 개수를 증가시키며 최적의 성능을 보이는 지점을 찾아가야함



### **3. [실습] 다음 지시에 따라 투표 기반 분류 모델을 만들어 보세요**
___

#### **STEP 1. MNIST 데이터를 불러들이고, 훈련, 검증, 테스트 데이터로 나누세요.**

In [2]:
# import MNIST dataset
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1, as_frame = False)
X, y = mnist["data"], mnist["target"]

In [3]:
# train/valid/test dataset
from sklearn.model_selection import train_test_split

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=10000, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=10000, random_state=42)

####  **STEP 2. 랜덤 포레스트 분류기, 엑스트라 트리 분류기, SVM 분류기, MLP 분류기를 훈련시키세요.**
- 모델 파라미터는 `n_estimators=100`, `random_state=42`로 설정합니다.

In [4]:
# import package
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

In [5]:
# classifier setting
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
et_clf = ExtraTreesClassifier(n_estimators=100, random_state=42)
svm_clf = LinearSVC(random_state=42) # SVC n_estimators 사용 X
mlp_clf = MLPClassifier(random_state=42) # MLP n_estimators 사용 X

# model fitting
rf_clf.fit(X_train, y_train)
et_clf.fit(X_train, y_train)
svm_clf.fit(X_train, y_train)
mlp_clf.fit(X_train, y_train)

MLPClassifier(random_state=42)

####  **STEP 3-1. 앞에서 훈련시킨 각 모델을 직접 투표 방법을 사용해 앙상블로 연결하고 훈련시킨 후, `score()`메서드를 이용하여 검증 데이터셋에서의 성능을 평가해보세요.**

In [6]:
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

In [8]:
# model fitting

estimators = [
    ('random_forest', rf_clf),
    ('extra_trees', et_clf),
    ('svm', svm_clf),
    ('mlp', mlp_clf)
]

voting_clf = VotingClassifier(estimators=estimators, voting='hard')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('random_forest',
                              RandomForestClassifier(random_state=42)),
                             ('extra_trees',
                              ExtraTreesClassifier(random_state=42)),
                             ('svm', LinearSVC(random_state=42)),
                             ('mlp', MLPClassifier(random_state=42))])

In [9]:
# model test
voting_clf.score(X_val, y_val)

0.9698

####  **STEP 3-2. 검증 데이터셋에서 각 분류 모델의 성능을 `score()` 메서드를 이용하여 확인해보고, 가장 성능이 낮은 모델을 제거하여 그 결과를 비교해보세요.**
- Hint : 가장 성능이 낮은 모델을 제거할 때 `del`를 활용해보세요

In [10]:
# 각 분류 모델 학습
for name, model in estimators:
    model.fit(X_train, y_train)

In [11]:
# 각 분류 모델의 성능 확인
scores = {}

for name, model in estimators:
    score = model.score(X_val, y_val)
    scores[name] = score
    print(f"{name} Accuracy: {score:.4f}")

random_forest Accuracy: 0.9692
extra_trees Accuracy: 0.9715
svm Accuracy: 0.9106
mlp Accuracy: 0.9606


- Q. 어떤 모델의 성능이 가장 낮나요?
- A. SVM

In [12]:
# 가장 성능이 낮은 모델 제거
del voting_clf.estimators[2]

In [13]:
# model fitting
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('random_forest',
                              RandomForestClassifier(random_state=42)),
                             ('extra_trees',
                              ExtraTreesClassifier(random_state=42)),
                             ('mlp', MLPClassifier(random_state=42))])

In [14]:
# 모델 제거 후 성능 확인
voting_clf.score(X_val, y_val)

0.9729

### **4. 다음 단계를 따라 앞에서 훈련시킨 분류 모델들을 이용하여 스태킹 앙상블을 구성해보자.**
___

#### **STEP 1. 3번 문제의 각 분류 모델을 실행해서 검증 세트에서 예측을 만들고, 그 결과로 훈련 세트를 만들어 보세요.**

In [15]:
X_val_predictions = np.empty((len(X_val), len(estimators)), dtype=np.float32)

In [18]:
for index, (_, estimator) in enumerate(estimators):
    X_val_predictions[:, index] = estimator.predict(X_val)

####  **STEP 2. 새로운 훈련 세트를 이용하여 랜덤 포레스트 분류 모델을 학습시켜 보세요.**

In [19]:
rf_blender = RandomForestClassifier(n_estimators=100, random_state=42)
rf_blender.fit(X_val_predictions, y_val)

RandomForestClassifier(random_state=42)

- 이 랜덤 포레스트 분류 모델이 바로 블렌더에 해당합니다.

####  **STEP 3. 이제 테스트셋에서 스태킹 앙상블 모델을 평가해보세요.**
- 성능 평가 지표로 **정확도**를 이용하세요.

In [21]:
# 각 분류 모델의 예측을 만들어 새로운 데이터셋 생성
X_test_predictions = np.empty((len(X_test), len(estimators)), dtype=np.float32)

for index, (_, estimator) in enumerate(estimators):
    X_test_predictions[:, index] = estimator.predict(X_test)

In [22]:
# 새로운 데이터셋을 이용하여 블렌더로 예측
y_pred = rf_blender.predict(X_test_predictions)

In [23]:
# model test
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9665